In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def centering_step(Q, p, A, b, t, v0, eps, alpha=0.1, beta=0.7):
    """
    Implements the centering step using Newton's method with logarithmic barriers and backtracking line search.

    Parameters:
        Q (numpy.ndarray): Positive semidefinite matrix of quadratic terms.
        p (numpy.ndarray): Linear term vector.
        A (numpy.ndarray): Constraint matrix.
        b (numpy.ndarray): Constraint vector.
        t (float): Barrier method parameter.
        v0 (numpy.ndarray): Initial variable.
        eps (float): Target precision.
        alpha (float): Line search parameter (typically small, e.g., 0.01).
        beta (float): Line search parameter (step size reduction factor, typically 0.5).

    Returns:
        list: Sequence of variable iterates (vi).
    """
    def objective(v):
        """Barrier objective function."""
        return t * (v.T @ Q @ v + p.T @ v) - np.sum(np.log(b - A @ v))

    def backtracking_line_search(v, dv, grad):
        """Backtracking line search to find step size."""
        step_size = 1.0
        while True:
            new_v = v + step_size * dv
            if np.all(b - A @ new_v > 0) and objective(new_v) <= objective(v) + alpha * step_size * grad.T @ dv:
                break
            step_size *= beta
        return step_size

    vi = [v0]
    v = v0
    i = 0
    while True and i < 100:
        gradient = t * (2 * Q @ v + p) + A.T @ (1 / (b - A @ v))

        D = np.diag(1 / (b - A @ v)**2)
        hessian = 2 * t * Q + A.T @ D @ A

        # Solve the Newton step (Hessian * dv = -grad)
        dv = np.linalg.solve(hessian, -gradient)

        decrement = np.sqrt(gradient.T @ -dv)

        # Check for convergence
        if decrement**2 / 2 <= eps:
            break

        step_size = backtracking_line_search(v, dv, gradient)

        v = v + step_size * dv

        vi.append(v)
        i += 1

    return vi

: 

In [ ]:
def barr_method(Q, p, A, b, v0, eps, mu=10, t0=1):
    """
    Implements the barrier method to solve QP using the centering_step function.

    Parameters:
        Q (numpy.ndarray): Positive semidefinite matrix of quadratic terms.
        p (numpy.ndarray): Linear term vector.
        A (numpy.ndarray): Constraint matrix.
        b (numpy.ndarray): Constraint vector.
        v0 (numpy.ndarray): Initial feasible point.
        eps (float): Precision criterion.
        mu (float): Scaling parameter for barrier method (e.g., mu > 1).
        t0 (float): Initial value of t (e.g., t0 > 0).

    Returns:
        list: Sequence of variable iterates (vi).
    """

    t = t0
    v = v0
    vi_seq = [v0]
    seq_t = [t0]


    while True:

        vi = centering_step(Q, p, A, b, t, v, eps)
        v = vi[-1]  # Final iterate of centering step
        vi_seq.extend(vi[1:])
        seq_t += [t] * len(vi[:-1])

        # Check for convergence
        precision_criterion = len(b) / t
        if precision_criterion <= eps:
            break

        t *= mu

    return vi_seq, seq_t

: 

In [ ]:
def test_barrier_method_dual(mu_values, Q, p, A, b, v0, eps):
    f_value_per_mu = []
    gap_per_mu = []      
    final_v_per_mu = [] 
    t_per_mu = []
    surrogate_f_star = None

    for mu in mu_values:
        vi_seq, seq_t = barr_method(Q, p, A, b, v0, eps, mu=mu)
        final_v = vi_seq[-1]
        final_v_per_mu.append(final_v)
        t_per_mu.append(seq_t)

        f_vt = [v.T @ Q @ v + p.T @ v for v in vi_seq]
        f_value_per_mu.append(f_vt)

        f_min = min(f_vt)
        if surrogate_f_star is None or f_min < surrogate_f_star:
            surrogate_f_star = f_min

    for f_vt in f_value_per_mu:
        gap_per_mu.append([f - surrogate_f_star for f in f_vt])

    return final_v_per_mu, f_value_per_mu, t_per_mu, gap_per_mu, surrogate_f_star

In [ ]:
def main():
    np.random.seed(42)
    n, d = 100, 50  
    X = np.random.randn(n, d)
    y = np.random.randn(n)
    lambda_ = 10
    m = 2 * d

    Q = 0.5 * np.eye(n)  
    p = -y  
    A = np.vstack([X.T, -X.T]) 
    b = lambda_ * np.ones(2 * d)  
    v0 = np.zeros(n)  
    eps = 1e-6

    mu_values = [2, 5, 15, 50, 100, 200, 500]
    final_v_per_mu, _ , t_per_mu, gap_per_mu, _ = test_barrier_method_dual(mu_values, Q, p, A, b, v0, eps)

    plt.figure(figsize=(12, 6))

    for i, mu in enumerate(mu_values):
        plt.semilogy(gap_per_mu[i], label=f"gap for μ = {mu}", color=f"C{i}")
        
        plt.semilogy(m / np.array(t_per_mu[i]), label=f"precision criterion for μ = {mu}", linestyle='dashed', color=f"C{i}")

    plt.xlabel("Iteration")
    plt.ylabel("Gap f(vt) - f*")
    plt.title("Convergence of Barrier Method for Different μ Values")
    plt.legend()
    plt.grid(True)
    plt.show()


    for i, mu in enumerate(mu_values):
        print(f"μ = {mu}, Final solution v: {final_v_per_mu[i]}")

    print("\nAn appropriate μ balances convergence speed and accuracy. Thus, the best μ value is 200.")

    return 0

: 

In [ ]:
main()

: 